In [5]:
import pandas as pd
import numpy as np
import keras
from keras import layers
import keras
from keras.applications import ResNet50V2
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [3]:
#data_path = "train.csv"
#data_df = pd.read_csv(data_path)

 ### Here, we instantiate a base model with pre-trained weights.

In [6]:
base_model = keras.applications.ResNet50V2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

2024-04-20 17:21:44.082337: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform XPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-20 17:21:44.082380: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:XPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: XPU, pci bus id: <undefined>)


94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


### Then, we freeze the base model.


In [7]:
base_model.trainable = False

### Create a new model on top of the pretrained model

In [8]:
inputs = keras.Input(shape=(150, 150, 3))
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

### Training the new model on blood clot data

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta=0.00001, # minimium amount of change to count as an improvement
    patience= 30, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [15]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [12]:
history = model.fit(
    X_train, y_train, #ADD the CORRECT df's for testing data
    validation_data=(X_valid,y_valid), #ADD the CORRECT df's for validation data
    batch_size=64,
    epochs=500,
    callbacks=[early_stopping], # put callbacks in a list
    verbose=0,  # turn off training log
)

### Plotting the loss history

In [ ]:
history_df = pd.DataFrame(history.history)#this creates a new df that keeps track of the training/validation 
history_df.loc[:, ['loss', 'val_loss']].plot();#this is used to visualize the loss history

print("Minimum validation loss: {}".format(history_df['val_loss'].min()))
print("Minimum training loss: {}".format(history_df['loss'].min()))

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are take into account
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

# Train end-to-end. Be careful to stop before you overfit!
model.fit(new_dataset, epochs=10, callbacks=..., validation_data=...)

In [17]:
## guide for adding it to the base model:
 Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are take into account
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

# Train end-to-end. Be careful to stop before you overfit!
model.fit(new_dataset, epochs=10, callbacks=..., validation_data=...)